# Image Segmentation for Road Detection

#### Individual's Name : Abhinav Raghav

####  Email : 200517685@georgianc.on.ca

### INTRODUCTION:
*********************************************************************************************************************
#### AIM : 
Automatically detecting buildings from satellite im- ages has a lot of potential applications, from monitoring move- ments of populations in remote areas to evaluating the avail- able surface to implant solar panels on roofs. We develop a Convolutional Neural Network for the extraction of buildings from satellite images, adapted from a U-net originally developed for biomedical image segmentation. We train our model on satellite images and on ground-truth labels extracted from OpenStreetMap. We show that our model achieves a reasonable level of accuracy, though slightly lower than state-of-the-art, and outline some ideas for further improvements.
*********************************************************************************************************************
#### Github Repo: 
https://github.com/abhinavr6293/Image_Segmentation_for_Road_Detection
*********************************************************************************************************************
#### DESCRIPTION OF PAPER:
Drawing out most recent layout of the populous areas in the city or a remote area can be laborious task if done outdated methods. These parts exist in metropolitan cities and are rapidly changing by the movement of humans or even in remote areas where attention of the authorities is minimum. During a natural disaster, it can be a promising tool in locating the damaged building of the area and can be helpful in finding routes to provide aide as early as possible. As in 2020 outbreak of COVID - 19 tested the readiness of the government, mapping out of hotspots and distribution of test
kits could have been fairly easy task by the likes of this project. Especially in a country like India, where people travel great distances from their homeland in search of work during a pandemic like this needed to be moved amidst the lockdown.
This project addresses the issue of segmentation of images obtained by satellite imaging. It dissects the image pixel by pixel and classifying each pixel as part of road.

*********************************************************************************************************************
#### PROBLEM STATEMENT :
This project addresses the broader issue of semantic segmentation of satellite images by aiming at classifying each pixel as belonging to a building or not. We developed a Convolutional Neural Network suitable for this task, inspired from the U-net [7]. We trained our model on a set of two-dimensional satellite images. The corresponding labels were binary masks, ie. two-dimensional matrices with ones for pixels where a building was present, zeros otherwise. Given a satellite image as input, our network was then able to output a corresponding predicted binary mask.
*********************************************************************************************************************
#### CONTEXT OF THE PROBLEM:
* The continually increasing number of complex datasets each year necessitates ever improving machine learning methods for robust and accurate categorization of these data.
* Generally, deep learning models involves a lot of randomization
* Users need to manually do hyper parameter tuning by changing each and every parameter which results into longer execution times
* So, They proposed an ensemble based approach for deep learning models.
*********************************************************************************************************************
#### SOLUTION:
In this project, we examined an end-to-end approach for semantic segmentation of satellite images for building de- tection using few data, and with a relatively low resolution. We implemented a CNN based on the U-net architecture developed by Ronneberger et al. in and used the MapBox API in OpenStreetMap to collect the datasets of interest. We used the high-level python API Keras to implement our model and facilitate data augmentation in order to improve its robustness. Our proposed approach achieved a reasonable accuracy, though slightly lower than state-of-the-art results published in the literature using other CNN architectures, and without post-processing. We then proposed a way to tackle the imprecise semantic segmentation of boundaries outputted by our model. Building upon this work and with the aim of preserving semantic boundaries, we plan to extend our network with further geometric cues and a uncertainty weighted multi- task loss, inspired by .


#### REFERENCES :
* [1] O. Benarchid and N. Raissouni, "support vector machines for object based building extraction in suburban area using very high resolution satellite images.," IAES International journal of AI, vol. 3, pp. 43-50, 2013.
* [2] B. Bischke, P. Helber, J. Folz and A. Damian Borth, "multi-task learning for segmentation of building footprints with deep neural network," in IEEE, 2015.
* [3] O. Ronneberger, P. F. and T. Brox, "U-Net: Convolutional Networks for biomedical image segmentation," in IEEE, 2015.
* [4] V. Mnih, "Machine Learning for Aerial Image Labeling," in University of Toronto, 2013.
* [5] E. Tiu, "Metrics to Evaluate your Semantic Segmentation Model," 9 august 2019.
* [6] Z. Yan, X. Han, C. Wang, Y. Qiu, Z. Xiong and S. Cu, "LEARNING MUTUALLY LOCAL-GLOBAL U-NETS FOR HIGH RESOLUTION RETINAL SEGMENTATION IN FUNDUS IMAGES," in IEEE, 2019.
* [7] H. Monajemi, D. L. Donoho and V. Stodden, "making massive computational experiment painlessly," in Proc. of IEEE Intl. Conf on Big Data, 2016.
* [8] Y. Chen, L. Cheng, M. Li, J. Wang, L. Tong and K. Yang, "Multiscale Grid Method for Detection and Reconstruction of rooftop usinf LiDAR Data," in IEEE, 2014.
* [9] G. Chho, C. B. Aramburu and I. Bougdal- Lambert, "Satellite Image Segmentation for Building Detection using U-Net," in IEEE, 2014.

## Architecture of the model :

* Instead of developing a model from scratch, we decided to use an existing model of Convolutional Neural Network for image segmentation. Namely, we turned to the U-net, originally developed for biomedical image segmentation [7]. Once trained, the network was able to output a pixel- wise binary classification (building or not) with good accuracy.

* Basically, the U-net builds upon the Fully Convolutional Network [4]. A contracting path extracts features of different levels through a sequence of convolutions, ReLU activations and max poolings, allowing to capture the context of each pixel. A symmetric expanding path then upsamples the result to increase the resolution of the detected features. In the U- net architecture, skip-connections (concatenations) are added between the contracting path and the expanding path, allowing precise localization as well as context. The expanding path therefore consists of a sequence of up-convolutions and concatenations with the corresponding feature map from the contracting path, followed by ReLU activations. The number of features is doubled at each level of downsampling. A figure of the U-net taken from [7] is presented below.




In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


# Implementation
********************************************************************************************************************
#### PART - 1: Data selection and train/dev/test splitting
#### PART - 2: Preprocessing
#### PART - 3: Data Augmentation

#### NECESSARY LIBRARIES
NOTE : import tensorflow as tf, import keras

### A. Data selection and train/dev/test splitting
We followed the advice of Drew Bollinger – a developer at DevelopmentSeed we met at the GISDay@Stanford – and collected our data using MapBox API for OpenStreetMap (OSM). OSM is an open-source mapping platform where any- one can edit maps, adding features such as roads or buildings. We ran the DevelopmentSeed Python API for Mapbox on Docker containers to scrap a dataset of satellite images, along with several corresponding layers (buildings, rivers, roads and woods) with pixel-labeled images for each one of these classes. As we were only interested in the extraction of buildings, we only downloaded the ”buildings” layer. We were able to acquire 34547 RGB images of size 256 × 256 × 3 from the territory of France, along with their corresponding RGB building mask.

In [3]:
import numpy as np
import pickle
import cv2
from tqdm import tqdm_notebook
import os
import json
import math
import h5py

In [4]:
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
%matplotlib inline

In [5]:
img_width = img_height = 256
num_channels = 3
root_path = "../Data/Road/"
mode = 'train'

In [8]:
def crop_images():
    """
    in order to maximize number of images to train our model we are croping available images and using them in order to
    increase sample size and preserve resolution
    """
    
    files = next(os.walk(root_path + mode + '/sat/'))[2]
    print('Total number of files =',len(files))
    
    for image_file in tqdm_notebook(files, total = len(files)):
       
   
        image_path = root_path+mode+'/sat/'+image_file
        image = cv2.imread(image_path)
        
        mask_path = root_path+mode+'/map/'+image_file[:-1]
        mask = cv2.imread(mask_path, 0)
        
        num_splits = math.floor((image.shape[0]*image.shape[1])/(img_width*img_height))
        counter = 0
        
        
        
        for r in range(0, image.shape[0], img_height):
            for c in range(0, image.shape[1], img_width):
                counter += 1
                blank_image = np.zeros((img_height ,img_width, 3), dtype = int)
                blank_mask = np.zeros((img_height ,img_width), dtype = int)
                
                new_image_path = root_path+mode+'/new/sat2/' + str(counter) + '_' + image_file
                new_mask_path = root_path+mode+'/new/map2/' + str(counter) + '_' + image_file
                
                new_image = np.array(image[r:r+img_height, c:c+img_width,:])
                new_mask = np.array(mask[r:r+img_height, c:c+img_width])
            
                
                blank_image[:new_image.shape[0], :new_image.shape[1], :] += new_image
                blank_mask[:new_image.shape[0], :new_image.shape[1]] += new_mask

                
                cv2.imwrite(new_image_path, blank_image)
                cv2.imwrite(new_mask_path, blank_mask)
                
crop_images()                  

Total number of files = 979


C:\Users\Abhishek\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [6]:
all_masks = []
all_images = []
def compress_images():
    """
    convert images and respective masks into a h5py file.
    """
    
    global all_images, all_masks
    rej_count = 0
    counter = 0
    
    files = next(os.walk(root_path + mode + '/new/sat2/'))[2]
    print('Total number of files =',len(files))
 

    for image_file in tqdm_notebook(files, total = len(files)):
        
        counter += 1
           
        
        image_path = root_path+mode+'/new/sat2/'+image_file   
        image = cv2.imread(image_path)
        
    
    
        mask_path = root_path+mode+'/new/map2/'+image_file
        mask = cv2.imread(mask_path, 0)
                  
        
        if (len(np.unique(mask)) == 1):
            rej_count += 1
            continue
        
        
        all_images.append(image)
        all_masks.append(mask)   
        
    
    all_images = np.asarray(all_images)
    all_masks = np.asarray(all_masks)
    print('{} images were rejected.'.format(rej_count))
    print("Shape of Train Images =", all_images.shape)
    print("Shape of Train Labels =", all_masks.shape)
    print("Memory size of Image array = ", all_images.nbytes)
    
    with h5py.File('../Data/Road/'+mode+'_images.h5py', 'w') as hf:
        hf.create_dataset("all_images",  data=all_images)
    
    with h5py.File('../Data/Road/'+mode+'_masks.h5py', 'w') as hf:
        hf.create_dataset("all_masks",  data=all_masks)
    
    print("Data has been successfully exported.")
    
compress_images()

Total number of files = 35244


C:\Users\Abhishek\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()



7656 images were rejected.
Shape of Train Images = (27588, 256, 256, 3)
Shape of Train Labels = (27588, 256, 256)
Memory size of Image array =  5424021504
Data has been successfully exported.
